In [4]:
import os
import tensorflow as tf 
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
print(tf.__version__)
tf.test.is_built_with_cuda()

2.1.0


True

In [6]:
model = Sequential()

model.add(Conv2D(input_shape=(128,128,3), filters=64, kernel_size=(3,3), padding='same', activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=3, activation="softmax"))

In [5]:
import h5py
from pathlib import Path
X = []
y = []
hdf5_dir = Path("F:\Dataset")
def read_hdf5data():    
    file = h5py.File(hdf5_dir / f"78819_dataset.h5", "r+")
    X = np.array(file["/images"]).astype("uint8")
    y = np.array(file["/meta"]).astype("uint8")
    return X, y

X,y = read_hdf5data()

print(X.shape)

(78819, 128, 128, 3)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

In [11]:
history = model.fit(x = X_train, y = y_train, validation_data = (X_test, y_test), batch_size = 64, validation_steps=350, epochs=3, callbacks=[checkpoint,early])

Train on 55173 samples, validate on 23646 samples
Epoch 1/3
55168/55173 [============================>.] - ETA: 0s - loss: 1.0987 - accuracy: 0.3351
Epoch 00001: val_accuracy did not improve from 0.33906
55173/55173 [==============================] - 1664s 30ms/sample - loss: 1.0987 - accuracy: 0.3351 - val_loss: 1.0407 - val_accuracy: 0.3324
Epoch 2/3
55168/55173 [============================>.] - ETA: 0s - loss: 1.0987 - accuracy: 0.3346
Epoch 00002: val_accuracy did not improve from 0.33906
55173/55173 [==============================] - 1659s 30ms/sample - loss: 1.0987 - accuracy: 0.3346 - val_loss: 1.0407 - val_accuracy: 0.3368
Epoch 3/3
55168/55173 [============================>.] - ETA: 0s - loss: 1.0987 - accuracy: 0.3326
Epoch 00003: val_accuracy did not improve from 0.33906
55173/55173 [==============================] - 1648s 30ms/sample - loss: 1.0987 - accuracy: 0.3326 - val_loss: 1.0408 - val_accuracy: 0.3324


In [2]:
import numpy as np
from tensorflow.keras.preprocessing import image
img = image.load_img("catreturn.jpeg", target_size=(128, 128))
img = np.array(img)
img = np.expand_dims(img, axis=0)

from tensorflow.keras.models import load_model
saved_model = load_model("vgg16_1.h5")

output = saved_model.predict(img)
print(output)

[[0.33387542 0.3281355  0.3379891 ]]
